### Zadanie: Klasyfikacja zwierząt morskich za pomocą modelu ResNet50

Twoim zadaniem jest stworzenie modelu sieci neuronowej, który będzie klasyfikować obrazy meduzy, ryby, rekina, tuńczyka, wieloryba. W tym celu wykorzystaj model ResNet50, który został wstępnie przeszkolony na szerokim zbiorze danych.

Kroki, które musisz podjąć:

1. **Załadowanie danych**: Dane treningowe i walidacyjne zostały już przygotowane w formie generatorów za pomocą biblioteki `ImageDataGenerator`. Generator treningowy zawiera obrazy treningowe, a generator walidacyjny zawiera obrazy walidacyjne. Obrazy są przeskalowane do rozmiaru `(224, 224)` i podzielone na dwie klasy: koty i psy.

2. **Model ResNet50**: W kodzie załadowano model ResNet50, który jest wstępnie przeszkolony na zestawie danych ImageNet. Model ten ma zamrożone wagi wszystkich warstw.

3. **Warstwy klasyfikacji**: Do modelu ResNet50 dodano warstwy klasyfikacji. Jest to warstwa `GlobalAveragePooling2D` i warstwa `Dense`, która wykorzystuje funkcję aktywacji softmax. Model ten jest przystosowany do zadania klasyfikacji, gdzie klasy to jelly, fish shark tuna whale.

4. **Kompilacja modelu**: Model jest skompilowany z optymalizatorem Adam, funkcją straty "categorical_crossentropy" (odpowiednia do klasyfikacji wieloklasowej), oraz metryką "accuracy".

5. **Trenowanie modelu**: Model jest trenowany na danych treningowych przez jedną epokę. Wynik trenowania jest wyświetlany na konsoli.

Twoim zadaniem jest uzupełnienie tego kodu o brakujące elementy, takie jak liczba epok trenowania, odblokowanie warstw modelu ResNet50 oraz dostosowanie modelu do odpowiedniego rodzaju klasyfikacji (wieloklasowej). Następnie możesz dostosować parametry trenowania, takie jak liczba epok czy wielkość wsadu (batch size), aby osiągnąć jak najlepsze wyniki w zadaniu klasyfikacji zwierząt morskich.

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Ustawienie ścieżki do katalogu z danymi
data_dir = '/kaggle/input/fall-23-intro-to-vision-dataset/Dataset/classification'

# Definicja generatora obrazów
datagen = ImageDataGenerator(
    rescale=1.0/255, # Przeskalowanie wartości pikseli do zakresu [0, 1]
    validation_split=0.2 # Procent danych przeznaczonych na zbiór walidacyjny
)

# Wczytanie danych treningowych
train_generator = datagen.flow_from_directory(
    '/kaggle/input/fall-23-intro-to-vision-dataset/Dataset/classification/train',
    target_size=(224, 224), # Rozmiar docelowy obrazów
    batch_size=32, # Rozmiar wsadu
    class_mode='categorical', # Rodzaj klasyfikacji (binary lub categorical)
    subset='training' # Ustawienie dla danych treningowych
)


2024-02-21 13:37:31.695978: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-21 13:37:31.696065: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-21 13:37:31.826481: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Found 931 images belonging to 5 classes.


In [2]:
# Wczytanie danych treningowych
validation_generator = datagen.flow_from_directory(
    '/kaggle/input/fall-23-intro-to-vision-dataset/Dataset/classification/val',
    target_size=(224, 224), # Rozmiar docelowy obrazów
    batch_size=32, # Rozmiar wsadu
    class_mode='categorical', # Rodzaj klasyfikacji (binary lub categorical)
    subset='training' # Ustawienie dla danych treningowych
)


Found 249 images belonging to 5 classes.


In [5]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in base_model.layers[:-1]:
    layer.trainable = False
    

# Adding new classification layers
model = Sequential([   base_model,   GlobalAveragePooling2D(),   Dense(5, activation='softmax')])
model.compile(optimizer=Adam(learning_rate=0.01), loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(train_generator, epochs=40, verbose=1, validation_data=validation_generator)

Epoch 1/40
30/30 [==============================] - 8s 175ms/step - loss: 1.5317 - accuracy: 0.4253 - val_loss: 1.1517 - val_accuracy: 0.5020
Epoch 2/40
30/30 [==============================] - 4s 139ms/step - loss: 1.2951 - accuracy: 0.4909 - val_loss: 1.1369 - val_accuracy: 0.5462
Epoch 3/40
30/30 [==============================] - 4s 137ms/step - loss: 1.2793 - accuracy: 0.4780 - val_loss: 1.0450 - val_accuracy: 0.6627
Epoch 4/40
30/30 [==============================] - 4s 141ms/step - loss: 1.1504 - accuracy: 0.5510 - val_loss: 1.0026 - val_accuracy: 0.6145
Epoch 5/40
30/30 [==============================] - 4s 139ms/step - loss: 1.0804 - accuracy: 0.5843 - val_loss: 0.9725 - val_accuracy: 0.6827
Epoch 6/40
30/30 [==============================] - 4s 140ms/step - loss: 1.0395 - accuracy: 0.5768 - val_loss: 0.9731 - val_accuracy: 0.6426
Epoch 7/40
30/30 [==============================] - 4s 138ms/step - loss: 1.0505 - accuracy: 0.5682 - val_loss: 0.9290 - val_accuracy: 0.6426
Epoch 

In [13]:
base_model.layers


### Zadanie prostsze: Analiza fake newsów na podstawie tytułów, 


Twoim zadaniem jest stworzenie modelu sieci neuronowej, który będzie klasyfikować tweety na podstawie tytułu. W danych mamy dostarczone tytuły artykułów oraz etykiety fake news (1) albo nie (0). 
Kroki, które musisz podjąć:

1. Przygotowanie danych: Dane zostały wczytane z pliku CSV. Teksty znajdują się w kolumnach title, a etykiety w kolumnie "class". Teksty oraz etykiety są przygotowane do dalszej obróbki.

2. Tokenizacja tekstu: Teksty powinny być tokenizowane za pomocą Tokenizer, a następnie przekształcane na sekwencje liczb. Jest to niezbędne, aby można było używać ich jako danych wejściowych do modelu sieci neuronowej.

3. Zbuduj model sieci neuronowej: Model sieci rekurencyjnej zdefiniuj w kodzie i przykładowo niech składa się z następujących warstw:

    * Warstwa wejściowa: Przyjmuje sekwencje tokenów o maksymalnej długości max_sequence_length.
    * Warstwa Embedding: Mapuje tokeny na wektory o wymiarze 100.
    * Warstwa LSTM: Warstwa rekurencyjna LSTM o rozmiarze 64, która przetwarza sekwencje wektorów.
    * Warstwa Dense: Warstwa wyjściowa z funkcją aktywacji softmax, która generuje dwa możliwe klasy.

4. Kompilacja modelu: Model jest kompilowany z optymalizatorem Adam, funkcją straty "sparse_categorical_crossentropy" (odpowiednia do klasyfikacji wieloklasowej) oraz metryką "accuracy".

5. Trenowanie modelu: Model jest trenowany na danych tweetów przez 5 epok, z wsadem o rozmiarze 32. Wynik trenowania jest oceniany na zbiorze walidacyjnym.

Twoim zadaniem jest uzupełnienie kodu i dostosowanie parametrów modelu oraz trenowania, aby uzyskać jak najlepsze wyniki w zadaniu analizy fake newsów na podstawie tytułów. Możesz eksperymentować z parametrami, takimi jak liczba epok trenowania, rozmiar wsadu (batch size), wymiar embeddingu czy rozmiar warstwy LSTM, aby dostosować model.


In [5]:
import pandas as pd 
import numpy as np 

fake = pd.read_csv("/kaggle/input/fake-news-detection/fake.csv")
true = pd.read_csv("/kaggle/input/fake-news-detection/true.csv")

fake['class']=0
true['class']=1


data = pd.concat([fake,true],axis=0)

In [6]:
data.head()

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [7]:
data['class']

0        0
1        0
2        0
3        0
4        0
        ..
21412    1
21413    1
21414    1
21415    1
21416    1
Name: class, Length: 44898, dtype: int64

In [8]:
X = data['title'].to_numpy()
y = data['class'].to_numpy()

In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
# Tokenizacja tekstu
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)
word_index = tokenizer.word_index

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(sequences, y, test_size=0.3, random_state=42)

max_sequence_length = max(len(seq) for seq in X)
X_train = pad_sequences(X_train, maxlen=max_sequence_length)
X_val = pad_sequences(X_val, maxlen=max_sequence_length)



In [7]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense,Dropout
from tensorflow.keras.optimizers import Adam
model = Sequential()
model.add(Embedding(input_dim=max_sequence_length, output_dim=100))
model.add(LSTM(64))
model.add(Dense(256))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, batch_size=500, verbose=1, validation_data=(X_val,y_val))


Epoch 1/10


I0000 00:00:1708452522.105430    7011 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


63/63 [==============================] - 13s 163ms/step - loss: 0.3418 - accuracy: 0.8572 - val_loss: 0.1698 - val_accuracy: 0.9327
Epoch 2/10
63/63 [==============================] - 8s 127ms/step - loss: 0.1574 - accuracy: 0.9401 - val_loss: 0.1541 - val_accuracy: 0.9405
Epoch 3/10
63/63 [==============================] - 6s 99ms/step - loss: 0.1467 - accuracy: 0.9435 - val_loss: 0.1478 - val_accuracy: 0.9405
Epoch 4/10
63/63 [==============================] - 3s 56ms/step - loss: 0.1485 - accuracy: 0.9432 - val_loss: 0.1495 - val_accuracy: 0.9420
Epoch 5/10
63/63 [==============================] - 5s 71ms/step - loss: 0.1378 - accuracy: 0.9481 - val_loss: 0.1435 - val_accuracy: 0.9431
Epoch 6/10
63/63 [==============================] - 4s 64ms/step - loss: 0.1329 - accuracy: 0.9488 - val_loss: 0.1432 - val_accuracy: 0.9434
Epoch 7/10
63/63 [==============================] - 3s 52ms/step - loss: 0.1308 - accuracy: 0.9497 - val_loss: 0.1396 - val_accuracy: 0.9439
Epoch 8/10
63/63 [===

Epoch 1/10
37/63 [================>.............] - ETA: 2s - loss: nan - accuracy: 0.5245

KeyboardInterrupt: 


### Zadanie trudniejsze: Analiza fake newsów na podstawie opisu, 


Twoim zadaniem jest stworzenie modelu sieci neuronowej, który będzie klasyfikować tweety na podstawie opisu. W danych mamy dostarczone artykuły oraz etykiety fake news (1) albo nie (0). 
Kroki, które musisz podjąć:

1. Przygotowanie danych: Dane zostały wczytane z pliku CSV. Teksty znajdują się w kolumnach title, a etykiety w kolumnie "class". Teksty oraz etykiety są przygotowane do dalszej obróbki.

2. Tokenizacja tekstu: Teksty powinny być tokenizowane za pomocą Tokenizer, a następnie przekształcane na sekwencje liczb. Jest to niezbędne, aby można było używać ich jako danych wejściowych do modelu sieci neuronowej.

3. Zbuduj model sieci neuronowej: Model sieci rekurencyjnej zdefiniuj w kodzie. 

4. Kompilacja modelu: Model jest kompilowany z optymalizatorem Adam, funkcją straty "sparse_categorical_crossentropy" (odpowiednia do klasyfikacji wieloklasowej) oraz metryką "accuracy".

5. Trenowanie modelu: Model jest trenowany na danych tweetów przez 5 epok, z wsadem o rozmiarze 32. Wynik trenowania jest oceniany na zbiorze walidacyjnym.

Twoim zadaniem jest uzupełnienie kodu i dostosowanie parametrów modelu oraz trenowania, aby uzyskać jak najlepsze wyniki w zadaniu analizy fake newsów na podstawie tytułów. Możesz eksperymentować z parametrami, takimi jak liczba epok trenowania, rozmiar wsadu (batch size), wymiar embeddingu czy rozmiar warstwy LSTM, aby dostosować model.


In [9]:
X = data['text'].to_numpy()
y = data['class'].to_numpy()

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
# Tokenizacja tekstu
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)
word_index = tokenizer.word_index

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(sequences, y, test_size=0.3, random_state=42)

max_sequence_length = 10000 #ile
X_train = pad_sequences(X_train, maxlen=max_sequence_length)
X_val = pad_sequences(X_val, maxlen=max_sequence_length)



2024-02-20 19:00:22.020678: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-20 19:00:22.020737: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-20 19:00:22.022488: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [10]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense,Dropout
from tensorflow.keras.optimizers import Adam
model = Sequential()
model.add(Embedding(input_dim=max_sequence_length, output_dim=100))
model.add(LSTM(64))
model.add(Dense(256))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, batch_size=10, verbose=1, validation_data=(X_val,y_val))

Epoch 1/10


I0000 00:00:1708455674.602040    5093 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


3143/3143 [==============================] - 996s 316ms/step - loss: 0.1175 - accuracy: 0.9583 - val_loss: 0.0601 - val_accuracy: 0.9830
Epoch 2/10
3143/3143 [==============================] - 849s 270ms/step - loss: 0.0333 - accuracy: 0.9910 - val_loss: 0.0400 - val_accuracy: 0.9924
Epoch 3/10
3143/3143 [==============================] - 837s 266ms/step - loss: 0.0208 - accuracy: 0.9942 - val_loss: 0.0362 - val_accuracy: 0.9895
Epoch 4/10
  20/3143 [..............................] - ETA: 11:22 - loss: 0.0150 - accuracy: 0.9950

Exception ignored in: <function AtomicFunction.__del__ at 0x7f8ae5319750>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/tensorflow/python/eager/polymorphic_function/atomic_function.py", line 286, in __del__
    def __del__(self):
KeyboardInterrupt: 


KeyboardInterrupt: 

In [11]:
3143/3143 [==============================] - 996s 316ms/step - loss: 0.1175 - accuracy: 0.9583 - val_loss: 0.0601 - val_accuracy: 0.9830
Epoch 2/10
3143/3143 [==============================] - 849s 270ms/step - loss: 0.0333 - accuracy: 0.9910 - val_loss: 0.0400 - val_accuracy: 0.9924
Epoch 3/10
3143/3143 [==============================] - 837s 266ms/step - loss: 0.0208 - accuracy: 0.9942 - val_loss: 0.0362 - val_accuracy: 0.9895

SyntaxError: invalid decimal literal (681889006.py, line 1)